Lung Cancer Knowledge Graph Pipeline

Complete ingestion + QA + analytics pipeline 

| Phase | Description |
|-------|-------------|
| **1** | Graph setup — uniqueness constraint + n10s graph config |
| **2** | Data ingestion — Wikidata · MeSH · Disease Ontology |
| **3** | Cross-ontology QA |
| **4** | Pattern detection — granularities · generalisations · triangles |
| **5** | Harmonisation suggestions |
| **6** | yFiles interactive visualisations |

> **Prerequisites**: Neo4j ≥ 5 with **APOC** and **neosemantics (n10s)** plugins.
> Credentials are read from `.env` in the project root.

---

In [1]:
# from yfiles_jupyter_graphs import GraphWidget
# GraphWidget()

In [2]:
import ipywidgets
ipywidgets.__version__

'8.1.8'

## 0 · Install & Import

In [3]:
import subprocess, sys

def _pip(*pkgs):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', *pkgs])

try:
    from dotenv import load_dotenv
except ImportError:
    _pip('python-dotenv')
    from dotenv import load_dotenv

try:
    from yfiles_jupyter_graphs import GraphWidget
except ImportError:
    _pip('yfiles_jupyter_graphs')
    from yfiles_jupyter_graphs import GraphWidget

try:
    import pandas as pd
except ImportError:
    _pip('pandas')
    import pandas as pd

import os
from pathlib import Path
from IPython.display import display
from neo4j import GraphDatabase

# Load .env from project root (one level above experiments/)
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
print('All imports OK')

All imports OK


## Neo4j Connection

In [4]:
NEO4J_URI      = os.getenv('NEO4J_URI',      'bolt://localhost:7687')
NEO4J_USER     = os.getenv('NEO4J_USER',     'neo4j')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD', '123456789')

driver  = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
driver.verify_connectivity()

# Persistent session – required for result.graph() used by GraphWidget
session = driver.session()

print(f'Connected to {NEO4J_URI}')

def run_cypher(query, params=None):
    """Execute a Cypher query and return all records as a list."""
    with driver.session() as s:
        return list(s.run(query, params or {}))

Connected to bolt://localhost:7687


---
## Phase 1 — Graph Setup

In [5]:
# ── Constraint (required by neosemantics) ─────────────────────────────────────
try:
    run_cypher("""
        CREATE CONSTRAINT n10s_unique_uri IF NOT EXISTS
        FOR (r:Resource) REQUIRE (r.uri) IS UNIQUE
    """)
    print('Constraint: OK')
except Exception as e:
    print(f'Constraint note: {e}')

# ── Graph config (required by neosemantics) ────────────────────────────────────
try:
    run_cypher("CALL n10s.graphconfig.init({ handleVocabUris: 'MAP' })")
    print('Graph config: OK')
except Exception as e:
    print(f'Graph config note: {e}')

Constraint: OK
Graph config note: {neo4j_code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `n10s.graphconfig.init`: Caused by: n10s.graphconfig.GraphConfigProcedures$GraphConfigException: The graph is non-empty. Config cannot be changed.} {gql_status: 52N37} {gql_status_description: error: procedure exception - procedure execution error. Execution of the procedure n10s.graphconfig.init() failed.}


---
## Phase 2 — Data Ingestion
### 2.1  Wikidata Lung Cancer Taxonomy

Imports **lung cancer and its subtypes** (`wd:Q12078`) from Wikidata via SPARQL CONSTRUCT.
Each disease node gets:
- label in English
- `HAS_PARENT` edges (subclass hierarchy)
- `SAME_AS` edges to MeSH and Disease Ontology when available

In [6]:
import time

_wd_count = run_cypher('MATCH (n:WD_Disease) RETURN count(n) AS c')[0]['c']
if _wd_count > 0:
    print(f'Wikidata already loaded ({_wd_count} WD_Disease nodes) — skipping import.')
else:
    # Optimised SPARQL: removed redundant parent check
    WD_SPARQL = '''PREFIX neo: <neo://voc#>
construct {
  ?dis a neo:WD_Disease ;
       neo:label ?disName ;
       neo:HAS_PARENT ?parentDisease ;
       neo:SAME_AS ?meshUri ;
       neo:SAME_AS ?diseaseOntoUri .
}
where {
  ?dis wdt:P279* wd:Q12078 ;
       rdfs:label ?disName . filter(lang(?disName) = "en")
  optional { ?dis wdt:P279 ?parentDisease . }
  optional { ?dis wdt:P486 ?meshCode .
             bind(URI(concat("http://id.nlm.nih.gov/mesh/",?meshCode)) as ?meshUri) }
  optional { ?dis wdt:P699 ?diseaseOntoId .
             bind(URI(concat("http://purl.obolibrary.org/obo/",REPLACE(?diseaseOntoId, ":", "_"))) as ?diseaseOntoUri) }
}
'''
    print('Importing Wikidata lung cancer taxonomy (may take a few minutes)...')

    # Retry up to 3 times with exponential backoff
    max_retries = 3
    for attempt in range(1, max_retries+1):
        try:
            wd_records = run_cypher(
                '''
                WITH $query AS query
                CALL n10s.rdf.import.fetch(
                  "https://query.wikidata.org/sparql?query=" + apoc.text.urlencode(query),
                  "N-Triples",
                  { headerParams: { Accept: "text/plain", `User-Agent`: "CoherencePLM-OntologyReconciliation/1.0 (research; neo4j-n10s)" } }
                )
                YIELD terminationStatus, triplesLoaded, triplesParsed, namespaces, extraInfo
                RETURN terminationStatus, triplesLoaded, triplesParsed, namespaces, extraInfo
                ''',
                {'query': WD_SPARQL}
            )
            for r in wd_records:
                d = dict(r)
                print(d)
                if d.get('terminationStatus') == 'KO':
                    raise Exception(f"Import failed: {d.get('extraInfo', '')}")
                else:
                    print(f'  Loaded {d.get("triplesLoaded", 0)} triples')
            break  # success
        except Exception as e:
            print(f"Attempt {attempt} failed: {e}")
            if attempt == max_retries:
                raise  # re-raise after final attempt
            wait = 2 ** attempt  # exponential backoff: 2, 4, 8 seconds
            print(f"Retrying in {wait} seconds...")
            time.sleep(wait)

Wikidata already loaded (4150 WD_Disease nodes) — skipping import.


### 2.1.1  Diagnostic – Check Wikidata Lung Cancer Labels

After import, let's see what labels contain 'lung' to understand the exact terminology used.

In [7]:
labels_with_lung = run_cypher("""
    MATCH (d:WD_Disease)
    WHERE d.label CONTAINS 'lung' OR d.label CONTAINS 'Lung'
    RETURN d.label AS label, d.uri AS uri LIMIT 20
""")
df_lung = pd.DataFrame([dict(r) for r in labels_with_lung])
print(f"Wikidata diseases containing 'lung': {len(df_lung)}")
if not df_lung.empty:
    display(df_lung.head(10))
else:
    print("No labels with 'lung' found. Check import or try different keywords.")

Wikidata diseases containing 'lung': 20


,label,uri
0,lung sarcoma,http://www.wikidata.org/entity/Q18555198
1,Limited-stage small cell lung carcinoma,http://www.wikidata.org/entity/Q65064515
2,lung hilum cancer,http://www.wikidata.org/entity/Q18557703
3,non-small-cell lung carcinoma,http://www.wikidata.org/entity/Q3658562
4,Combined small-cell lung carcinoma,http://www.wikidata.org/entity/Q5150915
5,asbestos-related lung carcinoma,http://www.wikidata.org/entity/Q18557665
6,Epithelial-myoepithelial carcinoma of the lung,http://www.wikidata.org/entity/Q5383692
7,lung non-squamous non-small cell carcinoma,http://www.wikidata.org/entity/Q63860017
8,Large-cell lung carcinoma with rhabdoid phenotype,http://www.wikidata.org/entity/Q6488976
9,large-cell lung carcinoma,http://www.wikidata.org/entity/Q6488975


### 2.2  QA — Explore Wikidata Taxonomy (Lung Cancer)

Now we try to find the taxonomy path for lung cancer. If exact match fails, we fall back to case‑insensitive search.

In [8]:
# Try exact match first, then fallback to contains
disease_name = 'lung cancer'
result = run_cypher("""
    MATCH (v:WD_Disease)
    WHERE v.label = $disease_name
    OPTIONAL MATCH path = (v)-[:HAS_PARENT*0..]->(root)
    WHERE NOT (root)-[:HAS_PARENT]->() OR root = v
    RETURN v, path
""", {'disease_name': disease_name})

if result:
    for r in result:
        if r['path']:
            path = r['path']
            print(f"Path found with {len(path.relationships)} hops:")
            for node in path.nodes:
                lbl = dict(node).get('label', dict(node).get('uri', '?'))
                print(f"    {sorted(node.labels)} → {lbl}")
        else:
            print(f"Node '{disease_name}' exists but has no parent hierarchy.")
else:
    print(f"No exact match for '{disease_name}'. Trying case‑insensitive contains...")
    result = run_cypher("""
        MATCH (v:WD_Disease)
        WHERE v.label =~ '(?i).*lung.*cancer.*'
        RETURN v.label, v.uri
    """)
    for r in result:
        print(f"Found: {r['v.label']} – {r['v.uri']}")

Node 'lung cancer' exists but has no parent hierarchy.
Path found with 1 hops:
    ['Resource', 'WD_Disease'] → lung cancer
    ['Resource'] → http://www.wikidata.org/entity/Q12136
Path found with 1 hops:
    ['Resource', 'WD_Disease'] → lung cancer
    ['Resource'] → http://www.wikidata.org/entity/Q3392853
Path found with 1 hops:
    ['Resource', 'WD_Disease'] → lung cancer
    ['Resource'] → http://www.wikidata.org/entity/Q15124212
Path found with 2 hops:
    ['Resource', 'WD_Disease'] → lung cancer
    ['Resource', 'WD_Disease'] → respiratory system cancer
    ['Resource'] → http://www.wikidata.org/entity/Q3286546
Path found with 5 hops:
    ['Resource', 'WD_Disease'] → lung cancer
    ['Resource', 'WD_Disease'] → respiratory system cancer
    ['Resource', 'WD_Disease'] → organ system cancer
    ['Resource', 'WD_Disease'] → cancer by anatomical entity
    ['Resource', 'WD_Disease'] → cancer
    ['Resource'] → http://www.wikidata.org/entity/Q12136
Path found with 5 hops:
    ['Resour

### 2.3  Remove Wikidata Shortcuts (Transitive Redundancies)

Shortcuts occur when a node `child` has both a **direct** `HAS_PARENT` to an ancestor `v`
**and** a longer path to the same ancestor. These create noise for path-length analysis.

In [9]:
wd_shortcuts = run_cypher("""
    MATCH shortcutPattern =
      (v:WD_Disease)<-[co:HAS_PARENT*2..]-(child)-[shortcut:HAS_PARENT]->(v)
    RETURN shortcutPattern LIMIT 2
""")
print(f'Shortcut examples found: {len(wd_shortcuts)}')

Shortcut examples found: 0


In [10]:
run_cypher("""
    MATCH (v:WD_Disease)<-[co:HAS_PARENT*2..]-(child)-[shortcut:HAS_PARENT]->(v)
    DELETE shortcut
""")
print('Wikidata shortcuts removed.')

Wikidata shortcuts removed.


### 2.4  MeSH Lung Cancer Taxonomy (Enhanced with Ancestors)

Imports MeSH descriptors under **Lung Neoplasms (D008175)**, including both narrower and broader terms to build complete hierarchies.

In [11]:
_mesh_count = run_cypher('MATCH (n:Mesh_Disease) RETURN count(n) AS c')[0]['c']
if _mesh_count > 0:
    print(f'MeSH already loaded ({_mesh_count} Mesh_Disease nodes) — skipping import.')
else:
    MESH_SPARQL = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX neo: <neo://voc#>

CONSTRUCT {
  ?s a neo:Mesh_Disease ;
       neo:label ?name ;
       neo:HAS_PARENT ?parentDescriptor .
}
FROM <http://id.nlm.nih.gov/mesh>
WHERE {
  { ?s meshv:broaderDescriptor* mesh:D008175 }   # descendants
  UNION
  { mesh:D008175 meshv:broaderDescriptor* ?s }   # ancestors (including self)
  ?s rdfs:label ?name .
  OPTIONAL { ?s meshv:broaderDescriptor ?parentDescriptor . }
}
'''
    print('Importing MeSH lung cancer taxonomy (with ancestors)...')
    mesh_records = run_cypher(
        '''
        WITH $query AS query
        CALL n10s.rdf.import.fetch(
          "https://id.nlm.nih.gov/mesh/sparql?format=TURTLE&query=" + apoc.text.urlencode(query),
          "Turtle"
        )
        YIELD terminationStatus, triplesLoaded, triplesParsed, namespaces, extraInfo
        RETURN terminationStatus, triplesLoaded, triplesParsed, namespaces, extraInfo
        ''',
        {'query': MESH_SPARQL}
    )
    for r in mesh_records:
        print(dict(r))

MeSH already loaded (18 Mesh_Disease nodes) — skipping import.


### 2.4.1  Diagnostic – Check MeSH Lung Neoplasms Labels

In [12]:
mesh_labels = run_cypher("""
    MATCH (m:Mesh_Disease)
    WHERE m.label CONTAINS 'Lung' OR m.label CONTAINS 'Neoplasms'
    RETURN m.label, m.uri LIMIT 20
""")
df_mesh = pd.DataFrame([dict(r) for r in mesh_labels])
print(f"MeSH diseases related to lung: {len(df_mesh)}")
if not df_mesh.empty:
    display(df_mesh.head(10))

MeSH diseases related to lung: 10


,m.label,m.uri
0,Adenocarcinoma of Lung,http://id.nlm.nih.gov/mesh/D000077192
1,Neoplasms by Site,http://id.nlm.nih.gov/mesh/D009371
2,Small Cell Lung Carcinoma,http://id.nlm.nih.gov/mesh/D055752
3,"Carcinoma, Non-Small-Cell Lung",http://id.nlm.nih.gov/mesh/D002289
4,Thoracic Neoplasms,http://id.nlm.nih.gov/mesh/D013899
5,Neoplasms,http://id.nlm.nih.gov/mesh/D009369
6,Lung Neoplasms,http://id.nlm.nih.gov/mesh/D008175
7,Lung Diseases,http://id.nlm.nih.gov/mesh/D008171
8,Respiratory Tract Neoplasms,http://id.nlm.nih.gov/mesh/D012142
9,Bronchial Neoplasms,http://id.nlm.nih.gov/mesh/D001984


In [13]:
# Remove MeSH shortcuts
run_cypher("""
    MATCH (v:Mesh_Disease)<-[co:HAS_PARENT*2..]-(child)-[shortcut:HAS_PARENT]->(v)
    DELETE shortcut
""")
print('MeSH shortcuts removed.')

MeSH shortcuts removed.


### 2.5  Disease Ontology (DO)

The DO is loaded from `doid.owl` (RDF/XML). We use **two-pass streaming**:
1. First pass collects URIs of `owl:Class` nodes.
2. Second pass imports only those triples relevant to those classes.

Mappings make the schema consistent with WD and MeSH:
- `owl:Class` → label `DO_Disease`
- `rdfs:subClassOf` → relationship `HAS_PARENT`

In [14]:
# ── Register namespace prefixes and semantic mappings ─────────────────────────
try:
    run_cypher('CALL n10s.nsprefixes.add("rdfs", "http://www.w3.org/2000/01/rdf-schema#")')
    print('rdfs prefix: OK')
except Exception as e:
    print(f'rdfs prefix note: {e}')

try:
    run_cypher('CALL n10s.mapping.add("http://www.w3.org/2000/01/rdf-schema#subClassOf", "HAS_PARENT")')
    print('subClassOf → HAS_PARENT mapping: OK')
except Exception as e:
    print(f'subClassOf mapping note: {e}')

try:
    run_cypher('CALL n10s.nsprefixes.add("owl", "http://www.w3.org/2002/07/owl#")')
    print('owl prefix: OK')
except Exception as e:
    print(f'owl prefix note: {e}')

try:
    run_cypher('CALL n10s.mapping.add("http://www.w3.org/2002/07/owl#Class", "DO_Disease")')
    print('owl:Class → DO_Disease mapping: OK')
except Exception as e:
    print(f'owl:Class mapping note: {e}')

rdfs prefix: OK
subClassOf → HAS_PARENT mapping: OK
owl prefix: OK
owl:Class → DO_Disease mapping: OK


In [15]:
_do_count = run_cypher('MATCH (n:DO_Disease) RETURN count(n) AS c')[0]['c']
if _do_count > 0:
    print(f'Disease Ontology already loaded ({_do_count} DO_Disease nodes) — skipping import.')
else:
    print('Loading Disease Ontology from purl.obolibrary.org (may take several minutes)...')
    do_result = run_cypher("""
        CALL n10s.rdf.stream.fetch(
          "http://purl.obolibrary.org/obo/doid.owl",
          "RDF/XML",
          { limit: 999999 }
        ) YIELD subject, predicate, object
        WHERE predicate = "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"
          AND object  = "http://www.w3.org/2002/07/owl#Class"
        WITH collect(subject) AS class_uris

        CALL n10s.rdf.stream.fetch(
          "http://purl.obolibrary.org/obo/doid.owl",
          "RDF/XML",
          { limit: 999999 }
        ) YIELD subject, predicate, object, isLiteral, literalType, literalLang, subjectSPO
        WHERE subject IN class_uris
          AND (
                predicate IN [
                  "http://www.w3.org/1999/02/22-rdf-syntax-ns#type",
                  "http://www.w3.org/2000/01/rdf-schema#label"
                ]
             OR predicate = "http://www.w3.org/2000/01/rdf-schema#subClassOf"
                AND n10s.rdf.isIRI(object)
             OR predicate = "http://www.geneontology.org/formats/oboInOwl#hasDbXref"
                AND object STARTS WITH "MESH:"
              )

        WITH n10s.rdf.collect.nt(
               subject, predicate, object,
               isLiteral, literalType, literalLang, subjectSPO
             ) AS taxonomy
        CALL n10s.rdf.import.inline(taxonomy, "N-Triples")
        YIELD terminationStatus, triplesLoaded, triplesParsed, namespaces, extraInfo, callParams
        RETURN terminationStatus, triplesLoaded, triplesParsed, namespaces, extraInfo, callParams
    """)
    for r in do_result:
        print(dict(r))

Disease Ontology already loaded (14521 DO_Disease nodes) — skipping import.


### 2.5.1  Diagnostic – Check DO Lung Cancer Labels

In [16]:
do_lung_labels = run_cypher("""
    MATCH (d:DO_Disease)
    WHERE d.label CONTAINS 'lung' OR d.label CONTAINS 'Lung'
    RETURN d.label, d.uri LIMIT 20
""")
df_do = pd.DataFrame([dict(r) for r in do_lung_labels])
print(f"DO diseases containing 'lung': {len(df_do)}")
if not df_do.empty:
    display(df_do.head(10))

DO diseases containing 'lung': 20


,d.label,d.uri
0,asbestos-related lung carcinoma,http://purl.obolibrary.org/obo/DOID_7596
1,lung carcinoma,http://purl.obolibrary.org/obo/DOID_3905
2,lung non-small cell carcinoma,http://purl.obolibrary.org/obo/DOID_3908
3,lung squamous cell carcinoma,http://purl.obolibrary.org/obo/DOID_3907
4,lung adenocarcinoma,http://purl.obolibrary.org/obo/DOID_3910
5,lung cancer,http://purl.obolibrary.org/obo/DOID_1324
6,lung combined large cell neuroendocrine carcinoma,http://purl.obolibrary.org/obo/DOID_7207
7,lung occult large cell carcinoma,http://purl.obolibrary.org/obo/DOID_7169
8,lung occult adenocarcinoma,http://purl.obolibrary.org/obo/DOID_7168
9,lung benign neoplasm,http://purl.obolibrary.org/obo/DOID_3683


### 2.6  Create SAME_AS Relationships from DO Cross-References

Disease Ontology stores MeSH cross-refs as node properties (`hasDbXref: "MESH:D000123"`).
We convert these to `SAME_AS` relationships pointing to MeSH URI nodes — consistent with
the Wikidata cross-references already in the graph.

In [17]:
link_result = run_cypher("""
    MATCH (doe:DO_Disease) WHERE doe.hasDbXref IS NOT NULL
    MERGE (mesh:Resource { uri: "http://id.nlm.nih.gov/mesh/" + substring(doe.hasDbXref, 5) })
    MERGE (doe)-[:SAME_AS]->(mesh)
    REMOVE doe.hasDbXref
    RETURN count(*) AS links_created
""")
print(f"SAME_AS links created: {link_result[0]['links_created'] if link_result else 0}")

SAME_AS links created: 0


### 2.7  Cross‑Ontology Link Summary

Let's count how many `SAME_AS` links exist between each pair of ontologies.

In [18]:
link_counts = run_cypher("""
    MATCH (a)-[:SAME_AS]-(b)
    RETURN labels(a) AS from, labels(b) AS to, count(*) AS count
    ORDER BY count DESC
""")
df_links = pd.DataFrame([dict(r) for r in link_counts])
print("SAME_AS link counts:")
if not df_links.empty:
    display(df_links)
else:
    print("No SAME_AS links found.")

SAME_AS link counts:


,from,to,count
0,[Resource],"[Resource, DO_Disease]",3973
1,"[Resource, DO_Disease]",[Resource],3973
2,"[Resource, DO_Disease]","[Resource, WD_Disease]",2351
3,"[Resource, WD_Disease]","[Resource, DO_Disease]",2351
4,"[Resource, WD_Disease]",[Resource],534
5,[Resource],"[Resource, WD_Disease]",534
6,"[Resource, DO_Disease]","[Resource, Mesh_Disease]",16
7,"[Resource, Mesh_Disease]","[Resource, DO_Disease]",16
8,"[Resource, WD_Disease]","[Resource, Mesh_Disease]",11
9,"[Resource, Mesh_Disease]","[Resource, WD_Disease]",11


---
## Phase 3 — Cross-Ontology QA
### 3.1  Wikidata ↔ MeSH Side-by-Side (Lung Cancer)

We'll use flexible matching to find lung cancer in both ontologies and compare hierarchies.

In [19]:
wd_path = run_cypher("""
    MATCH path = (v:WD_Disease)-[:HAS_PARENT*0..]->(root)
    WHERE v.label =~ '(?i).*lung.*cancer.*'
      AND NOT (root)-[:HAS_PARENT]->() OR root = v
    RETURN [n IN nodes(path) | n.label] AS wd_hierarchy
    LIMIT 3
""")

mesh_path = run_cypher("""
    MATCH path = (v:Mesh_Disease)-[:HAS_PARENT*0..]->(root)
    WHERE v.label =~ '(?i).*lung.*neoplasms.*'
      AND NOT (root)-[:HAS_PARENT]->() OR root = v
    RETURN [n IN nodes(path) | n.label] AS mesh_hierarchy
    LIMIT 3
""")

print("Wikidata hierarchy:", wd_path[0]['wd_hierarchy'] if wd_path else "Not found")
print("MeSH hierarchy:", mesh_path[0]['mesh_hierarchy'] if mesh_path else "Not found")

Wikidata hierarchy: ['Giant-cell glioblastoma']
MeSH hierarchy: ['Pancoast Syndrome']


### 3.2  Sibling Matches (DO ↔ MeSH)

Find DO and MeSH concepts that share a common parent – this indicates close alignment even if exact matches are missing.

In [20]:
siblings = run_cypher("""
    MATCH (do:DO_Disease)-[:HAS_PARENT]->(parentDO)<-[:HAS_PARENT]-(do2:DO_Disease)
    MATCH (mesh:Mesh_Disease)-[:SAME_AS]-(do)
    MATCH (mesh2:Mesh_Disease)-[:SAME_AS]-(do2)
    WHERE mesh <> mesh2
    RETURN do.label AS do1, do2.label AS do2, mesh.label AS mesh1, mesh2.label AS mesh2,
           parentDO.label AS commonParent
    LIMIT 20
""")
df_siblings = pd.DataFrame([dict(r) for r in siblings])
print(f"Sibling matches found: {len(df_siblings)}")
if not df_siblings.empty:
    display(df_siblings)

Sibling matches found: 8


,do1,do2,mesh1,mesh2,commonParent
0,pulmonary blastoma,Pancoast tumor,Pulmonary Blastoma,Pancoast Syndrome,lung cancer
1,lung non-small cell carcinoma,lung small cell carcinoma,"Carcinoma, Non-Small-Cell Lung",Small Cell Lung Carcinoma,lung carcinoma
2,Pancoast tumor,pulmonary blastoma,Pancoast Syndrome,Pulmonary Blastoma,lung cancer
3,malignant pleural mesothelioma,pulmonary blastoma,"Mesothelioma, Malignant",Pulmonary Blastoma,pleural disease
4,pulmonary blastoma,malignant pleural mesothelioma,Pulmonary Blastoma,"Mesothelioma, Malignant",pleural disease
5,lung small cell carcinoma,lung non-small cell carcinoma,Small Cell Lung Carcinoma,"Carcinoma, Non-Small-Cell Lung",lung carcinoma
6,bronchial benign neoplasm,lung benign neoplasm,Bronchial Neoplasms,Lung Neoplasms,respiratory system benign neoplasm
7,lung benign neoplasm,bronchial benign neoplasm,Lung Neoplasms,Bronchial Neoplasms,respiratory system benign neoplasm


### 3.3  Orphan Detection

Find Wikidata diseases with no cross‑ontology links – these are candidates for enrichment.

In [21]:
orphans = run_cypher("""
    MATCH (wd:WD_Disease)
    WHERE NOT (wd)-[:SAME_AS]-()
    RETURN wd.label, wd.uri LIMIT 10
""")
df_orphans = pd.DataFrame([dict(r) for r in orphans])
print(f"Wikidata diseases with no cross‑references: {len(df_orphans)}")
if not df_orphans.empty:
    display(df_orphans)

Wikidata diseases with no cross‑references: 10


,wd.label,wd.uri
0,Giant-cell glioblastoma,http://www.wikidata.org/entity/Q3772317
1,Precursor T-lymphoblastic lymphoma,http://www.wikidata.org/entity/Q7239591
2,metastatic colon cancer,http://www.wikidata.org/entity/Q108566365
3,Lennert lymphoma,http://www.wikidata.org/entity/Q6522771
4,burn scar carcinoma,http://www.wikidata.org/entity/Q2663329
5,ameloblastic carcinoma,http://www.wikidata.org/entity/Q25099685
6,papillary glioneuronal tumor,http://www.wikidata.org/entity/Q1440338
7,HPV-positive oropharyngeal cancer,http://www.wikidata.org/entity/Q5635621
8,Mycosis fungoides involving spleen,http://www.wikidata.org/entity/Q55015601
9,Ameloblastic fibroma,http://www.wikidata.org/entity/Q4742342


---
## Phase 4 — Pattern Detection
### Pattern 1 — Different Granularities (DO ↔ MeSH)

Finds pairs of DO concepts cross-linked to MeSH where the **path length differs** between
the two ontologies — indicating a mismatch in hierarchical depth.

In [22]:
p1 = run_cypher("""
    MATCH topLink    = (topDo:DO_Disease)-[:SAME_AS]-(topMesh:Mesh_Disease)
    MATCH bottomLink = (bottomDo:DO_Disease)-[:SAME_AS]-(bottomMesh:Mesh_Disease)
    MATCH txnDo      = (topDo)<-[:HAS_PARENT*]-(bottomDo)
    MATCH txnMesh    = (topMesh)<-[:HAS_PARENT*]-(bottomMesh)
    WHERE length(txnDo) <> length(txnMesh)
    RETURN * LIMIT 1
""")
print(f'Pattern 1 (different granularities): {len(p1)} example(s) found')

Pattern 1 (different granularities): 1 example(s) found


### Pattern 2 — Generalisations

A single MeSH concept is cross-linked to **multiple** DO (or WD) concepts that are
related by `HAS_PARENT`. This signals that one ontology is more coarse-grained.

In [23]:
# DO ↔ MeSH generalisations
p2a = run_cypher("""
    MATCH multiXRef = (md1:DO_Disease)-[:SAME_AS]-(start:Mesh_Disease)-[:SAME_AS]-(md2:DO_Disease)
    OPTIONAL MATCH link = (md1)-[r:HAS_PARENT*]->(md2)
    RETURN multiXRef, link LIMIT 20
""")
print(f'Pattern 2 — DO ↔ MeSH generalisations: {len(p2a)} row(s)')

Pattern 2 — DO ↔ MeSH generalisations: 2 row(s)


In [24]:
# WD ↔ MeSH generalisations
p2b = run_cypher("""
    MATCH multiXRef = (md1:WD_Disease)-[:SAME_AS]-(start:Mesh_Disease)-[:SAME_AS]-(md2:WD_Disease)
    OPTIONAL MATCH link = (md1)-[r:HAS_PARENT*]->(md2)
    RETURN multiXRef, link LIMIT 20
""")
print(f'Pattern 2 — WD ↔ MeSH generalisations: {len(p2b)} row(s)')

Pattern 2 — WD ↔ MeSH generalisations: 0 row(s)


### Pattern 3 — Perfect Triangles

A **triangle** is when one disease concept in all three ontologies cross-links to
each of the other two and each node has exactly 2 `SAME_AS` relationships (no fan-out).

In [25]:
p3 = run_cypher("""
    MATCH triangle = (wdid:WD_Disease)-[:SAME_AS]-(do:DO_Disease)
                     -[:SAME_AS]-(md:Mesh_Disease)-[:SAME_AS]-(wdid)
    WHERE size([path = (wdid)-[:SAME_AS]-() | path]) =
          size([path = (do)-[:SAME_AS]-()   | path]) =
          size([path = (md)-[:SAME_AS]-()   | path]) = 2
    RETURN triangle LIMIT 50
""")
print(f'Pattern 3 (perfect triangles): {len(p3)} found (capped at 50)')

Pattern 3 (perfect triangles): 9 found (capped at 50)


### 4.4  Missing WD‑MeSH Suggestions (via DO)

If perfect triangles are rare, we can still suggest links where a WD node and a MeSH node are both connected to the same DO node but not to each other.

In [26]:
suggestions = run_cypher("""
    MATCH (wd:WD_Disease)-[:SAME_AS]-(do:DO_Disease)-[:SAME_AS]-(mesh:Mesh_Disease)
    WHERE NOT (wd)-[:SAME_AS]-(mesh)
    RETURN wd.label AS wd_label, wd.uri AS wd_uri,
           mesh.label AS mesh_label, mesh.uri AS mesh_uri
    LIMIT 20
""")
df_suggest = pd.DataFrame([dict(r) for r in suggestions])
print(f"Missing WD‑MeSH links (via DO): {len(df_suggest)}")
if not df_suggest.empty:
    display(df_suggest)

Missing WD‑MeSH links (via DO): 4


,wd_label,wd_uri,mesh_label,mesh_uri
0,malignant pleural mesothelioma,http://www.wikidata.org/entity/Q18557602,"Mesothelioma, Malignant",http://id.nlm.nih.gov/mesh/D000086002
1,mesothelioma,http://www.wikidata.org/entity/Q1077603,"Mesothelioma, Malignant",http://id.nlm.nih.gov/mesh/D000086002
2,cancer,http://www.wikidata.org/entity/Q12078,Neoplasms,http://id.nlm.nih.gov/mesh/D009369
3,lung benign neoplasm,http://www.wikidata.org/entity/Q18556020,Lung Neoplasms,http://id.nlm.nih.gov/mesh/D008175


---
## Phase 5 — Harmonisation Suggestions

Detects **incomplete triangles** where WD ↔ DO and DO ↔ MeSH links exist but
WD ↔ MeSH is missing. Outputs RDF triples ready to enrich Wikidata (property P486).

In [27]:
harm = run_cypher("""
    MATCH incomplete = (wdid:WD_Disease)-[:SAME_AS]-(do:DO_Disease)-[:SAME_AS]-(md:Mesh_Disease)
    WHERE NOT (md)-[:SAME_AS]-(wdid)
      AND size([(wdid)-[:SAME_AS]-() | 1]) = 1
      AND size([(md)-[:SAME_AS]-()   | 1]) = 1
      AND size([(do)-[:SAME_AS]-()   | 1]) = 2
    RETURN wdid.uri AS subject,
           'http://www.wikidata.org/prop/direct/P486' AS predicate,
           n10s.rdf.getIRILocalName(md.uri) AS object
""")

df_harm = pd.DataFrame([dict(r) for r in harm])
print(f'Harmonisation suggestions: {len(df_harm)}')
df_harm.head(20) if not df_harm.empty else print('No suggestions found.')

Harmonisation suggestions: 2


,subject,predicate,object
0,http://www.wikidata.org/entity/Q12078,http://www.wikidata.org/prop/direct/P486,D009369
1,http://www.wikidata.org/entity/Q18556020,http://www.wikidata.org/prop/direct/P486,D008175


---
## Phase 6 — yFiles Interactive Visualisations

### Helper Utilities

Colour legend:

| Colour | Ontology |
|--------|----------|
| 🔵 Blue (`#4e9af1`) | Wikidata (`WD_Disease`) |
| 🟢 Green (`#50c878`) | MeSH (`Mesh_Disease`) |
| 🟠 Orange (`#ff8c00`) | Disease Ontology (`DO_Disease`) |
| ⚪ Grey | Generic resource |

In [28]:
import warnings

# ── Colour palette (one colour per ontology source) ───────────────────────────
ONTOLOGY_COLORS = {
    'WD_Disease':   '#4e9af1',   # Wikidata          – blue
    'Mesh_Disease': '#50c878',   # MeSH              – emerald green
    'DO_Disease':   '#ff8c00',   # Disease Ontology  – amber
}

# ──────────────────────────────────────────────────────────────────────────────
# show_graph  – mirrors the reference notebook pattern exactly
#   widget = GraphWidget(graph = session.run(cypher).graph())
#
# GraphWidget(graph=…) overwrites the node 'label' with combined Neo4j labels
# ("WD_Disease:Resource"), so we pre-capture human-readable names from the raw
# Neo4j nodes BEFORE creating the widget, then restore them via node_label_mapping.
# ──────────────────────────────────────────────────────────────────────────────
def show_graph(cypher, params=None, title='', layout='hierarchic', directed=True):
    """
    Display a Cypher query result as an interactive yFiles graph.

    Parameters
    ----------
    cypher   : str   – the Cypher query (paths / nodes / rels)
    params   : dict  – optional query parameters
    title    : str   – header printed above the widget
    layout   : str   – 'hierarchic' | 'organic' | 'circular' | 'tree' | 'radial'
    directed : bool  – show arrow-heads on edges
    """
    result = session.run(cypher, params or {})
    graph  = result.graph()

    # ── 1. Capture names & colours BEFORE widget overwrites 'label' ───────────
    name_lookup  = {}   # node internal-id → human-readable disease name
    color_lookup = {}   # node internal-id → hex colour
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', DeprecationWarning)
        for n in graph.nodes:
            nid  = n.id          # integer – matches what GraphWidget stores as node['id']
            props = dict(n)
            name  = props.get('label', '')
            uri   = props.get('uri', '')
            name_lookup[nid]  = name or (uri.rstrip('/').split('/')[-1] if uri else str(nid))
            for lbl in ('WD_Disease', 'Mesh_Disease', 'DO_Disease'):
                if lbl in n.labels:
                    color_lookup[nid] = ONTOLOGY_COLORS[lbl]
                    break

    if not name_lookup:
        print(f'[{title or "graph"}]  Query returned no nodes.')
        return

    # ── 2. Build the widget exactly like the reference notebook ───────────────
    w = GraphWidget(graph=graph)
    w.set_directed(directed)

    # ── 3. Node label: restore disease name via mapping ───────────────────────
    w.node_label_mapping = lambda index, node: \
        name_lookup.get(node.get('id', index), node['properties'].get('label', str(index)))

    # ── 4. Node colour: one colour per ontology ───────────────────────────────
    w.node_color_mapping = lambda index, node: \
        color_lookup.get(node.get('id', index), '#cccccc')

    # ── 5. Edge label: relationship type ─────────────────────────────────────
    w.edge_label_mapping = lambda index, edge: \
        edge.get('properties', {}).get('label', '')

    # ── 6. Layout (direct method call, exactly as in reference notebook) ──────
    layout_methods = {
        'hierarchic': w.hierarchic_layout,
        'organic':    w.organic_layout,
        'circular':   w.circular_layout,
        'tree':       w.tree_layout,
        'radial':     w.radial_layout,
    }
    layout_methods.get(layout, w.hierarchic_layout)()

    # ── 7. Print summary and display once ────────────────────────────────────
    if title:
        print(f'=== {title} ===')
    print(f'  Nodes: {len(w.nodes)}  |  Edges: {len(w.edges)}')
    display(w)          # display once; do NOT return w (avoids double-render)

print('yFiles show_graph helper ready  (v' + __import__("yfiles_jupyter_graphs").__version__ + ')')

yFiles show_graph helper ready  (v1.10.10)


### Viz 1 — Wikidata Taxonomy: Lung Cancer

Shows the full `HAS_PARENT` chain from *lung cancer* up to the root in Wikidata.
Layout: **hierarchic** (top-down tree).

In [29]:
show_graph(
    """
    MATCH taxonomy = (v:WD_Disease)-[:HAS_PARENT*]->(root)
    WHERE v.label =~ '(?i).*lung.*cancer.*'
      AND NOT (root)-[:HAS_PARENT]->()
    RETURN taxonomy
    """,
    title='Wikidata taxonomy — lung cancer',
    layout='hierarchic',
)

=== Wikidata taxonomy — lung cancer ===
  Nodes: 34  |  Edges: 39


GraphWidget(layout=Layout(height='800px', width='100%'))

### Viz 2 — Cross-Ontology: Wikidata ↔ MeSH (Lung Cancer)

Each WD node in the taxonomy is joined to its MeSH twin via `SAME_AS`.
Blue = Wikidata, Green = MeSH.  Layout: **organic** (force-directed).

In [30]:
show_graph(
    """
    MATCH taxonomy = (v:WD_Disease)-[:HAS_PARENT*]->(root)
    WHERE v.label =~ '(?i).*lung.*cancer.*'
      AND NOT (root)-[:HAS_PARENT]->()
    UNWIND nodes(taxonomy) AS node
    MATCH mesh_twin = (node)-[:SAME_AS*0..1]->(:Mesh_Disease)
                       -[:HAS_PARENT*0..]->(mesh_root:Mesh_Disease)
    WHERE NOT (mesh_root)-[:HAS_PARENT]->()
    RETURN taxonomy, mesh_twin
    """,
    title='Cross-ontology: Wikidata ↔ MeSH — lung cancer',
    layout='organic',
)

=== Cross-ontology: Wikidata ↔ MeSH — lung cancer ===
  Nodes: 38  |  Edges: 43


GraphWidget(layout=Layout(height='800px', width='100%'))

### Viz 3 — Lung Neoplasms Cross-References in MeSH

In [31]:
show_graph(
    """
    MATCH ln = (md:Mesh_Disease)-[:SAME_AS]-()
    WHERE md.label =~ '(?i).*lung.*neoplasms.*'
    RETURN ln
    """,
    title='Lung Neoplasms — MeSH cross-references (SAME_AS)',
    layout='organic',
    directed=False,
)

=== Lung Neoplasms — MeSH cross-references (SAME_AS) ===
  Nodes: 2  |  Edges: 1


GraphWidget(layout=Layout(height='500px', width='100%'))

### Viz 4 — Pattern 2: Generalisations (DO ↔ MeSH)

Shows a MeSH hub node cross-linked to multiple DO concepts that are hierarchically related.
Layout: **organic**.

In [32]:
show_graph(
    """
    MATCH multiXRef = (md1:DO_Disease)-[:SAME_AS]-(start:Mesh_Disease)-[:SAME_AS]-(md2:DO_Disease)
    OPTIONAL MATCH link = (md1)-[r:HAS_PARENT*]->(md2)
    RETURN multiXRef, link LIMIT 20
    """,
    title='Pattern 2 — DO ↔ MeSH generalisations (top 20)',
    layout='organic',
)

=== Pattern 2 — DO ↔ MeSH generalisations (top 20) ===
  Nodes: 3  |  Edges: 3


GraphWidget(layout=Layout(height='500px', width='100%'))

### Viz 5 — Pattern 2: Generalisations (WD ↔ MeSH)

In [33]:
# First, check if there are any MeSH nodes linked to more than one WD_Disease
hub_count = run_cypher("""
    MATCH (mesh:Mesh_Disease)-[:SAME_AS]-(wd:WD_Disease)
    WITH mesh, count(wd) AS wd_count
    WHERE wd_count >= 2
    RETURN count(mesh) AS hubs
""")[0]['hubs']

if hub_count == 0:
    print("\n❌ No MeSH hub linking multiple Wikidata diseases found.")
    print("   This means there are no 'generalisations' where one MeSH concept")
    print("   corresponds to two different Wikidata diseases that are hierarchically related.")
    print("   → However, there are 11 direct WD–MeSH links. You can explore them below.")
    print("   → For generalisation patterns, look at DO–MeSH (Viz 4) – they exist.\n")
    
    # Show existing WD–MeSH links as a simple graph
    show_graph(
        """
        MATCH path = (wd:WD_Disease)-[:SAME_AS]-(mesh:Mesh_Disease)
        RETURN path LIMIT 30
        """,
        title='Existing WD–MeSH direct links',
        layout='organic',
        directed=False,
    )
else:
    show_graph(
        """
        MATCH multiXRef = (md1:WD_Disease)-[:SAME_AS]-(start:Mesh_Disease)-[:SAME_AS]-(md2:WD_Disease)
        OPTIONAL MATCH link = (md1)-[r:HAS_PARENT*]->(md2)
        RETURN multiXRef, link LIMIT 20
        """,
        title='Pattern 2 — WD ↔ MeSH generalisations (top 20)',
        layout='organic',
    )


❌ No MeSH hub linking multiple Wikidata diseases found.
   This means there are no 'generalisations' where one MeSH concept
   corresponds to two different Wikidata diseases that are hierarchically related.
   → However, there are 11 direct WD–MeSH links. You can explore them below.
   → For generalisation patterns, look at DO–MeSH (Viz 4) – they exist.

=== Existing WD–MeSH direct links ===
  Nodes: 22  |  Edges: 11


GraphWidget(layout=Layout(height='720px', width='100%'))

### Viz 6 — Pattern 3: Perfect Triangles (WD ↔ DO ↔ MeSH)

Each triangle is a disease perfectly matched across all three ontologies.
Layout: **circular** (shows the triangle structure clearly).

In [34]:
show_graph(
    """
    MATCH triangle = (wdid:WD_Disease)-[:SAME_AS]-(do:DO_Disease)
                     -[:SAME_AS]-(md:Mesh_Disease)-[:SAME_AS]-(wdid)
    WHERE size([path = (wdid)-[:SAME_AS]-() | path]) =
          size([path = (do)-[:SAME_AS]-()   | path]) =
          size([path = (md)-[:SAME_AS]-()   | path]) = 2
    RETURN triangle LIMIT 50
    """,
    title='Pattern 3 — Perfect triangles WD ↔ DO ↔ MeSH (up to 50)',
    layout='circular',
    directed=False,
)

=== Pattern 3 — Perfect triangles WD ↔ DO ↔ MeSH (up to 50) ===
  Nodes: 27  |  Edges: 27


GraphWidget(layout=Layout(height='770px', width='100%'))

### Viz 7 — Harmonisation Candidates

Incomplete triangles where the WD ↔ MeSH link is missing.
Each cluster of 3 nodes should become a triangle after the suggested enrichment.
Layout: **organic** (clusters naturally).

In [35]:
show_graph(
    """
    MATCH p1 = (wdid:WD_Disease)-[:SAME_AS]-(do:DO_Disease),
          p2 = (do)-[:SAME_AS]-(md:Mesh_Disease)
    WHERE NOT (md)-[:SAME_AS]-(wdid)
      AND size([(wdid)-[:SAME_AS]-() | 1]) = 1
      AND size([(md)-[:SAME_AS]-()   | 1]) = 1
      AND size([(do)-[:SAME_AS]-()   | 1]) = 2
    RETURN p1, p2 LIMIT 30
    """,
    title='Harmonisation candidates — missing WD ↔ MeSH links (up to 30)',
    layout='organic',
    directed=False,
)

=== Harmonisation candidates — missing WD ↔ MeSH links (up to 30) ===
  Nodes: 6  |  Edges: 4


GraphWidget(layout=Layout(height='500px', width='100%'))